# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv("Weather.csv")
weather_df.head()



,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,Kichera,RU,55.94,110.10,58.41,70,99,5.48,1597009147
1,1,Port Lincoln,AU,-34.73,135.87,50.76,89,5,12.95,1597009148
2,2,Rikitea,PF,-23.12,-134.97,69.94,76,28,23.44,1597009149
3,3,Lompoc,US,34.64,-120.46,71.60,60,11,12.75,1597009150
4,4,Carnarvon,AU,-24.87,113.63,64.40,68,40,13.87,1597009151


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [15]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
humidity = weather_df["Humidity"].astype(float)

# Graphing the heat map with various parameters
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,max_intensity=100,
                                 point_radius = 10, dissipating=True)

# Adding the heat layer to the visual
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
#Selecting the ideal conditions for vacation using loc.
conditions_df=weather_df.loc[(weather_df["Max Temp"] <90) & (weather_df["Max Temp"] >=80) & (weather_df["Wind Speed"] <10) & (weather_df["Cloudiness"] == 0) & (weather_df["Humidity"] <50)]

#Making sure the number of location is less than 10
print(len(conditions_df))

#Removing the extra column
del conditions_df["Unnamed: 0"]
conditions_df.head()

2


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
196,Nova Granada,BR,-20.53,-49.31,80.60,18,0,5.82,1597009358
398,Mayor Pablo Lagerenza,PY,-19.93,-60.77,85.51,23,0,5.95,1597009580


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
#Storing hotel_df as the main datafram
hotel_df=conditions_df

#Creating an empty list to store hotel names
hotels=[]


#Creating loop and setting up parameters to target hotel names for the given ideal condition cities
for index,row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat=row["Lat"]
    lng=row["Lng"]
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "type": "hotel",
        "key": g_key}
    
    response= requests.get(base_url, params=params)
    response=response.json()
    
#Appending the hotel names into the empty list
    hotels.append(response["results"][0]["name"])

#Adding a column with the hotel names
hotel_df["Hotel Name"]=hotels    
hotel_df.head()

C:\Users\nguye\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
196,Nova Granada,BR,-20.53,-49.31,80.60,18,0,5.82,1597009358,Nova Granada
398,Mayor Pablo Lagerenza,PY,-19.93,-60.77,85.51,23,0,5.95,1597009580,Lagerenza 4 de Mayo


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
hotel_layer=gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=10,
    info_box_content=hotel_info)

# Display figure
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))